In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext("local", "main")

In [3]:
sc.textFile("/Users/jshuster/projects/personal/kaggle/data/0ACDbR5M3ZhBJajygTuf.bytes")

/Users/jshuster/projects/personal/kaggle/data/0ACDbR5M3ZhBJajygTuf.bytes MappedRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [11]:
rdd = sc.textFile("/Users/jshuster/projects/personal/kaggle/data/test_bytes/fileList.txt")

In [62]:
rdd.repartition(2)

MappedRDD[51] at repartition at NativeMethodAccessorImpl.java:-2

In [72]:
pairRdd = rdd.flatMap(lambda x:
    [("one", [1,0,0]), ("two",[0,1,0]), ("three",[0,0,1])]
)

In [2]:
n = 4
c = 9

def add_to_ngram_map(filename,ngram,ngram_map):
    ''' convenience function to add an ngram to the ngram_map '''
    label_idx = fname2label[file_name]-1
    if not ngram in ngram_map:
        ngram_map[ngram] = [0]*c
    ngram_map[ngram][label_idx] += 1
    
def ngram_count(filenames):
    ''' given a set of files, returns a map of ngrams->frequencies '''
    ngram_map = {}
    for filename in filenames:
        byte_list = []   
        # read each line, appending bytes (pairs of hex chars) to a list
        with open(filename, "r") as f:
            rows = f.readlines()
            for row in rows:
                byte_list+=row.split()[1:]
                byte_list[-1].rstrip('\n') # get rid of the annoying return lines
        
        ngram_seq = byte_list[:n] # get the first n-gram of the file
        ngram = ngram_seq.join()
        add_to_ngram_map(file_name,ngram,ngram_map)
 
        for byte in byte_list[n:]:
            
            # get the next ngram
            ngram_seq = ngram_seq[1:]
            ngram_seq += byte
            ngram = ngram_seq.join()
            
            # add it to the map
            add_to_ngram_map(file_name,ngram,ngram_map)
    
    return ngram_map

In [73]:
pairRdd.collect()

[('one', [1, 0, 0]),
 ('two', [0, 1, 0]),
 ('three', [0, 0, 1]),
 ('one', [1, 0, 0]),
 ('two', [0, 1, 0]),
 ('three', [0, 0, 1])]

In [33]:
rdd

/Users/jshuster/projects/personal/kaggle/data/test_bytes/fileList.txt MappedRDD[11] at textFile at NativeMethodAccessorImpl.java:-2

In [70]:
def accumulate(k, x):
    return [i+j for i,j in zip(k,x)]

In [44]:
accumulate([[2,0],[0,1]])

[2, 1, 0]

In [71]:
pairRdd.reduceByKey(accumulate).collect()

[('one', [2, 0, 0]), ('three', [0, 0, 2]), ('two', [0, 2, 0])]

In [82]:
rdd.flatMap(NgramCount).collect()

[('00526600 00 00 00 00', [1]),
 ('00582FF0 ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ?? ??\r\n', [1])]

In [87]:
def bargraphRuined(x):
    accum = 0
    for i in x:
        accum += 1
    return [("one", [accum,0,0]), ("two",[0,1,0]), ("three",[0,0,1])]

In [88]:
testRdd = rdd.mapPartitions(bargraphRuined)

In [89]:
testRdd.collect()

[('one', [2, 0, 0]), ('two', [0, 1, 0]), ('three', [0, 0, 1])]

In [90]:
rdd.collect()

[u'/Users/jshuster/projects/personal/kaggle/data/test_bytes/0A32eTdBKayjCWhZqDOQ.bytes',
 u'/Users/jshuster/projects/personal/kaggle/data/test_bytes/0ACDbR5M3ZhBJajygTuf.bytes']

In [93]:
testRdd = rdd.repartition(2).mapPartitions(bargraphRuined)

In [94]:
testRdd.reduceByKey(accumulate).collect()

[('one', [2, 0, 0]), ('three', [0, 0, 2]), ('two', [0, 2, 0])]